In [64]:
import sys
import re
from os import walk
from pathlib import Path
from functools import reduce

In [13]:
ROOT_PATH = "/Users/janschill/code/rails/rails"

In [14]:
def full_path(file_name):
    return ROOT_PATH + file_name

In [6]:
# Number of lines
def LOC(file):
    return sum([1 for line in open(file)])

In [ ]:
def extract_from_file(name, line):
    x = re.search("^" + name + " (\S+)", line) 

In [7]:
# Number of methos
def NOM(file):
    pass

In [9]:
def extract_require_from_line(line):
    print(line)
    x = re.search("^require (\S+)", line) 
    print(x)
    return x.group(1)

In [110]:
test_file = full_path('lib/kafka/message_buffer.rb')
lines = [line for line in open(test_file)]
for line in lines[:10]:
    # print(line)
    print(extract_require_from_line(line))


# frozen_string_literal: true

None


AttributeError: 'NoneType' object has no attribute 'group'

In [90]:
def requires(file):
    # extracts all the imported modules from a file
    lines = [line for line in open(file)]
    
    all_requires = []
    for line in lines:
        try:
            all_requires.append(extract_require_from_line(line))
        except:
            continue

    return all_requires

test_file = full_path('lib/kafka/message_buffer.rb')
assert "kafka/protocol/message" in requires(test_file)

AssertionError: 

In [29]:
directories = []
for (dirpath, dirnames, filenames) in walk(ROOT_PATH):
    directories.extend(dirnames)
    break
directories = list(filter(lambda x: x[0] != ".", directories))

In [59]:
def get_files(path, file_extension):
    files = Path(path).rglob("*." + file_extension)
    # Print all Ruby files from the Repo
    ruby_files = []
    for file in files:
        ruby_files.append({'filename': str(file), 'LOC': LOC(file)})
    return ruby_files

In [75]:
def reduce_loc(list):
    sum = 0
    for el in list:
        sum = sum + el['LOC']
    return sum

In [81]:
rails_modules = {}
for directory in directories:
    files = get_files(ROOT_PATH + '/' + directory + '/', 'rb')
    average_LOC = int(reduce_loc(files) / len(files))
    rails_modules[directory] = { 'files': files, 'average LOC': average_LOC }
rails_modules = {k: v for k, v in sorted(rails_modules.items(), key = lambda item: item[0])}

In [117]:
from tabulate import tabulate
format = "{:<10}{:<1}{:<6}{:<1}{:<10}"
print(format.format("Ruby files", " | ", "Ø LOC", " | ", "Module"))
print('-----------+--------+--------------')
for k, v in rails_modules.items():
    print(format.format(len(v['files']), ' | ', v['average LOC'], ' | ', k))

Ruby files | Ø LOC  | Module    
-----------+--------+--------------
86         | 80     | actioncable
93         | 34     | actionmailbox
41         | 140    | actionmailer
325        | 202    | actionpack
74         | 35     | actiontext
186        | 231    | actionview
118        | 70     | activejob 
130        | 118    | activemodel
853        | 160    | activerecord
124        | 62     | activestorage
469        | 119    | activesupport
1          | 22     | ci        
22         | 69     | guides    
278        | 148    | railties  
1          | 323    | tasks     
2          | 21     | tools     
